In [2]:
import mysql.connector
from path import *
from mysql.connector import Error as MySqlError
from bson.json_util import loads
from constants import *
import os

### Connection 

In [3]:
PW = 'root'

def create_db_connection(host_name, user_name, user_password, db_name):
	connection = mysql.connector.connect(
		host=host_name,
		user=user_name,
		passwd=user_password,
		database=db_name
	)
	return connection

CONNECTION = create_db_connection("localhost", "root", PW, 'maadb_project')

### Utils

In [6]:
def retrieve_data_from_json(path):
	with open(path, 'r', encoding='utf8') as f:
		return loads(f.read())

def exec_query(query):
	cursor = CONNECTION.cursor()
	cursor.execute(query)
	CONNECTION.commit()
	

In [5]:
sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'hate', 'hope', 'joy', 'like', 'love', 'sadness', 'surprise', 'trust', 'pos', 'neg']
COMPLETED_OP = "Operazione riuscita con successo"

#### Creazione tabella sentiment

In [ ]:
#exec_query("drop table maadb_project.sentiment", connection)
query = """CREATE TABLE `maadb_project`.`sentiment` (
			`nome` VARCHAR(20) NOT NULL, 
			PRIMARY KEY (`nome`));"""
exec_query(query)

#### Inserimento dati tabella sentiment

In [ ]:
def insert_sentiments():
	query = """INSERT INTO `maadb_project`.`sentiment` (`nome`)"""
	insert = """VALUES ('{}'),"""
	for s in sentiments:
		query += insert.format(s)

	exec_query(query)

insert_sentiments()

#### Creazione tabella lexical_resource

In [ ]:
#exec_query("drop table maadb_project.lexical_resource", connection)
query = """CREATE TABLE `maadb_project`.`lexical_resource` (
			`id_lex_res` INT NOT NULL AUTO_INCREMENT,
			`sentiment` VARCHAR(20) NOT NULL,
			`resource_name` VARCHAR(10) NOT NULL,
			`number_of_words` INT NOT NULL,
			PRIMARY KEY (`id_lex_res`),
			CONSTRAINT `sentiment_lexres`
			FOREIGN KEY (`sentiment`) REFERENCES `maadb_project`.`sentiment` (`nome`)
			ON DELETE RESTRICT ON UPDATE CASCADE);
		"""
exec_query(query)

#### Inserimento dati tabella lexical_resources

In [ ]:
def insert_lexical_resources():
	path = f'{PREP_PATH}/LexResources.json'
	lex_res = retrieve_data_from_json(path)
	query = """INSERT INTO `maadb_project`.`lexical_resource` (`sentiment`, `resource_name`, `number_of_words`)"""
	insert = """VALUES ('{}', '{}', '{}'),"""
	for item in lex_res:
		query += insert.format(item["sentiment"], item["id"], item["totNumberWords"])
	print("Lexical Resources correctly inserted")

insert_lexical_resources()

#### Creazione tabella word

In [ ]:
#exec_query("drop table maadb_project.word", connection)
query = """CREATE TABLE `maadb_project`.`word` (
			`id_word` INT NOT NULL AUTO_INCREMENT,
			`lemma` VARCHAR(20) NOT NULL,
			PRIMARY KEY (`id_word`));"""
exec_query(query)

#### Inserimento dati tabella word

#### Creazione tabella token

In [ ]:
#exec_query("drop table maadb_project.token", connection)
query = """CREATE TABLE `maadb_project`.`token` (
			`id_token` INT NOT NULL AUTO_INCREMENT,
			`token` VARCHAR(20) NOT NULL,
			`type` VARCHAR(20) NOT NULL,
			PRIMARY KEY (`id_token`));"""
exec_query(query)

#### Inserimento dati tabella token

In [ ]:
def insert_token_of_sent_on_db(path):
    data = retrieve_data_from_json(path)
    insert_non_words_query = "INSERT INTO token (token, type) VALUES"
    insert_single_quote = '\n("{}", "{}"),'
    insert_double_quote = "\n('{}', '{}'),"
    for tweet in data:
        for type in ['emojis', 'emoticons']:
            for token in tweet[type]:
                if token.find('"') > 0:
                    insert_non_words_query += insert_double_quote.format(token, type)
                else:
                    insert_non_words_query += insert_single_quote.format(token, type)
    insert_non_words_query = insert_non_words_query[:-1] + ";"
    exec_query(insert_non_words_query)

def insert_token_on_db():
    for sent in sentiments:
        path = os.path.join(PREP_PATH, f"preprocessed_tweets_{sent}.json")
        if os.path.exists(path):
            insert_token_of_sent_on_db(path)
    print("Non-word tokens correctly inserted")

insert_token_on_db()

#### Creazione tabella tweet

In [ ]:
#exec_query("drop table maadb_project.tweet", connection)
query = """CREATE TABLE `maadb_project`.`tweet` (
			`id_tweet` INT NOT NULL AUTO_INCREMENT,
			`sentiment` VARCHAR(20) NOT NULL,
			`doc_number` INT NOT NULL,
			PRIMARY KEY (`id_tweet`),
			CONSTRAINT `sentiment_tweet`
			FOREIGN KEY (`sentiment`) REFERENCES `maadb_project`.`sentiment` (`nome`)
			ON DELETE RESTRICT ON UPDATE CASCADE);"""
exec_query(query)


#### Inserimento dati tabella tweet

In [ ]:
def insert_tweet():
	insert = "('{}', '{}'),"
	for s in sentiments:
		query = """INSERT INTO `maadb_project`.`tweet` (`sentiment`, `doc_number`) VALUES"""
		for i in range(60000):
			query += insert.format(s, i)
		exec_query(query[:-1] + ";")
		print(f"Completed {s}")
insert_tweet()

#### Creazione tabella wordsintweet

In [ ]:
#exec_query("drop table maadb_project.wordsintweet", connection)
query = """CREATE TABLE `maadb_project`.`wordsintweet` (
			`id_word` INT NOT NULL,
			`id_tweet` INT NOT NULL,
			`frequence` INT NOT NULL,
			`pos_tag` VARCHAR(4) NULL,
			PRIMARY KEY (`id_word`, `id_tweet`),
			CONSTRAINT `tweet_id`
				FOREIGN KEY (`id_tweet`) REFERENCES `maadb_project`.`tweet` (`id_tweet`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `word_id`
				FOREIGN KEY (`id_word`) REFERENCES `maadb_project`.`word` (`id_word`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati tabella wordsintweet

#### Creazione tabella tokensintweet

In [ ]:
#exec_query("drop table maadb_project.tokenintweet", connection)
query = """CREATE TABLE `maadb_project`.`tokensintweet` (
			`id_token` INT NOT NULL,
			`id_tweet` INT NOT NULL,
			`frequence` INT NOT NULL,
			PRIMARY KEY (`id_token`, `id_tweet`),
			CONSTRAINT `tweet_token_id`
				FOREIGN KEY (`id_tweet`) REFERENCES `maadb_project`.`tweet` (`id_tweet`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `token_id`
				FOREIGN KEY (`id_token`) REFERENCES `maadb_project`.`token` (`id_token`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati in tabella tokensintweet

#### Creazione tabella wordsinlexres

In [ ]:
#exec_query("drop table maadb_project.wordsinlexres", connection)
query = """CREATE TABLE `maadb_project`.`wordsinlexres` (
			`id_word` INT NOT NULL,
			`id_lexres` INT NOT NULL,
			PRIMARY KEY (`id_word`, `id_lexres`),
			CONSTRAINT `id_word_lexres`
				FOREIGN KEY (`id_word`) REFERENCES `maadb_project`.`word` (`id_word`)
				ON DELETE CASCADE ON UPDATE CASCADE,
			CONSTRAINT `id_lexres`
				FOREIGN KEY (`id_lexres`) REFERENCES `maadb_project`.`lexical_resource` (`id_lex_res`)
				ON DELETE CASCADE ON UPDATE CASCADE);"""
exec_query(query)

#### Inserimento dati in tabella wordsinlexres